##  Aggregations


### Step 1: Initialize PySpark Session


In [24]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


### Step 2: Load the Dataset


In [25]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/US_Crime_Rates_1960_2014.csv"  # Replace with the actual path
US_Crime_Rates_1960_2014_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)


In [26]:
US_Crime_Rates_1960_2014_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Violent: integer (nullable = true)
 |-- Property: integer (nullable = true)
 |-- Murder: integer (nullable = true)
 |-- Forcible_Rape: integer (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated_assault: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny_Theft: integer (nullable = true)
 |-- Vehicle_Theft: integer (nullable = true)



In [27]:
US_Crime_Rates_1960_2014_df.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

In [28]:
US_Crime_Rates_1960_2014_df.createOrReplaceTempView("crime_rates")

### count

Question: How many records are there in the US_Crime_Rates_1960_2014_df DataFrame?

In [29]:
#pyspark

row_count= US_Crime_Rates_1960_2014_df.count()
print("Number of records =", row_count)

Number of records = 55


In [30]:
#sql

len1= spark.sql('''select count(*)
               from crime_rates''')

print("Number of records:", len1.collect()[0][0])

Number of records: 55


### countDistinct
Question: How many distinct years are present in the US_Crime_Rates_1960_2014_df DataFrame?
Answer:

In [31]:
#pyspark
from pyspark.sql.functions import countDistinct

#distinct_years= US_Crime_Rates_1960_2014_df.distinct().count()
distinct_years= US_Crime_Rates_1960_2014_df.select(countDistinct("Year"))
print("No. of distinct years= ", distinct_years.collect()[0][0])

No. of distinct years=  55


In [32]:
#sql 

distinct_year_count= spark.sql('''select count(distinct year) 
from crime_rates''')

print("No. of distinct years = ", distinct_year_count.collect()[0][0])

No. of distinct years =  55


### approx_count_distinct

Question: Estimate the approximate number of distinct values in the "Total" column of the US_Crime_Rates_1960_2014_df DataFrame.

In [33]:
#pyspark
from pyspark.sql.functions import approx_count_distinct

total= US_Crime_Rates_1960_2014_df.select(approx_count_distinct("Total",0.1))
print("Approximate distinct values in 'Total' column: ",total.collect()[0][0])

Approximate distinct values in 'Total' column:  51


In [34]:
#sql

total_dist_values= spark.sql('''select approx_count_distinct(Total,0.1) 
                             from crime_rates''')
print("Approximate distinct values in 'Total' column: ",total_dist_values.collect()[0][0])

Approximate distinct values in 'Total' column:  51


###  first and last

Question: Find the first and last year in the US_Crime_Rates_1960_2014_df DataFrame.

In [35]:
#pyspark    

from pyspark.sql.functions import first, last

first_and_last_values= US_Crime_Rates_1960_2014_df.select(first("Year"), last("Year"))

print("first year = " ,first_and_last_values.collect()[0][0])
print("first year = " ,first_and_last_values.collect()[0][1])

first year =  1960
first year =  2014


In [36]:
#sql

f_and_l_values= spark.sql('''select first(Year), last(Year)
                          from crime_rates''')

print("first year = " ,f_and_l_values.collect()[0][0])
print("first year = " ,f_and_l_values.collect()[0][1])

first year =  1960
first year =  2014


### min and max

Question: Find the minimum and maximum population values in the US_Crime_Rates_1960_2014_df DataFrame.

In [37]:
#pyspark    

from pyspark.sql.functions import min, max

min_max = US_Crime_Rates_1960_2014_df.select(min("Population"), max("Population"))

print("Minimun population = ",min_max.collect()[0][0])
print("Maximun population = ", min_max.collect()[0][1])

Minimun population =  179323175
Maximun population =  318857056


### sumDistinct

Question: Calculate the sum of distinct "Property" values for each year in the US_Crime_Rates_1960_2014_df DataFrame.

In [38]:
#pyspark
from pyspark.sql.functions import sum_distinct

US_Crime_Rates_1960_2014_df.groupBy("Year").agg(sum_distinct("Property").alias("SumDistinctProperty")).show()


+----+-------------------+
|Year|SumDistinctProperty|
+----+-------------------+
|1990|           12655500|
|1975|           10252700|
|1977|            9955000|
|2003|           10442862|
|2007|            9843481|
|1974|            9278700|
|2006|            9983568|
|1978|           10123400|
|1961|            3198600|
|2013|            8650761|
|1988|           12356900|
|1997|           11558175|
|1994|           12131900|
|1968|            6125200|
|2014|            8277829|
|1973|            7842200|
|1979|           11041500|
|1971|            7771700|
|1966|            4793300|
|2004|           10319386|
+----+-------------------+
only showing top 20 rows



In [39]:
#sql

sum_of_distinct_property = spark.sql('''select year,
                            sum(distinct property) as SumDistinctProperty
                            from crime_rates
                            group by year''')

sum_of_distinct_property.show()

+----+-------------------+
|year|SumDistinctProperty|
+----+-------------------+
|1990|           12655500|
|1975|           10252700|
|1977|            9955000|
|2003|           10442862|
|2007|            9843481|
|1974|            9278700|
|2006|            9983568|
|1978|           10123400|
|1961|            3198600|
|2013|            8650761|
|1988|           12356900|
|1997|           11558175|
|1994|           12131900|
|1968|            6125200|
|2014|            8277829|
|1973|            7842200|
|1979|           11041500|
|1971|            7771700|
|1966|            4793300|
|2004|           10319386|
+----+-------------------+
only showing top 20 rows



### avg

Question: Calculate the average "Murder" rate for the entire dataset in the US_Crime_Rates_1960_2014_df DataFrame.
Answer:

In [40]:
#pyspark
from pyspark.sql.functions import avg

avg_murder= US_Crime_Rates_1960_2014_df.select(avg("Murder"))

print("Average murder rate: ", avg_murder.collect()[0][0])


Average murder rate:  17317.236363636363


In [41]:
#sql

avg_murder_rate = spark.sql('''select avg(murder)
                            from  crime_rates''') 
print("Average murder rate: ",avg_murder_rate.collect()[0][0])

Average murder rate:  17317.236363636363


### Aggregating to Complex Types

Question: Calculate the total sum of "Violent" and "Property" crimes for each year in the US_Crime_Rates_1960_2014_df DataFrame. Store the results in a struct type column.

In [42]:
#pyspark
from pyspark.sql.functions import struct, col, sum 

total_sum= US_Crime_Rates_1960_2014_df.groupBy("Year").agg(
    struct(
        sum(col("Violent")).alias("TotalViolent"),
        sum(col("Property")).alias("TotalProperty")
    ).alias("CrimeSums ")
)
total_sum.show()

+----+-------------------+
|Year|         CrimeSums |
+----+-------------------+
|1990|{1820130, 12655500}|
|1975|{1039710, 10252700}|
|1977| {1029580, 9955000}|
|2003|{1383676, 10442862}|
|2007| {1408337, 9843481}|
|1974|  {974720, 9278700}|
|2006| {1418043, 9983568}|
|1978|{1085550, 10123400}|
|1961|  {289390, 3198600}|
|2013| {1199684, 8650761}|
|1988|{1566220, 12356900}|
|1997|{1634770, 11558175}|
|1994|{1857670, 12131900}|
|1968|  {595010, 6125200}|
|2014| {1197987, 8277829}|
|1973|  {875910, 7842200}|
|1979|{1208030, 11041500}|
|1971|  {816500, 7771700}|
|1966|  {430180, 4793300}|
|2004|{1360088, 10319386}|
+----+-------------------+
only showing top 20 rows



In [43]:
#sql

sum_total= spark.sql('''select Year,
       struct(sum(Violent) AS TotalViolent, sum(Property) AS TotalProperty) as 
       CrimeSum
from crime_rates
group by Year
order by year
''')
sum_total.show()

+----+-------------------+
|Year|           CrimeSum|
+----+-------------------+
|1960|  {288460, 3095700}|
|1961|  {289390, 3198600}|
|1962|  {301510, 3450700}|
|1963|  {316970, 3792500}|
|1964|  {364220, 4200400}|
|1965|  {387390, 4352000}|
|1966|  {430180, 4793300}|
|1967|  {499930, 5403500}|
|1968|  {595010, 6125200}|
|1969|  {661870, 6749000}|
|1970|  {738820, 7359200}|
|1971|  {816500, 7771700}|
|1972|  {834900, 7413900}|
|1973|  {875910, 7842200}|
|1974|  {974720, 9278700}|
|1975|{1039710, 10252700}|
|1976|{1004210, 10345500}|
|1977| {1029580, 9955000}|
|1978|{1085550, 10123400}|
|1979|{1208030, 11041500}|
+----+-------------------+
only showing top 20 rows



### Grouping

Question: In the given US_Crime_Rates_1960_2014_df DataFrame, you are tasked with finding the average of all crimes combined for each year. Calculate the sum of all crime categories (Violent, Property, Murder, Forcible_Rape, Robbery, Aggravated_assault, Burglary, Larceny_Theft, Vehicle_Theft) for each year and then determine the average of these combined crime sums. Provide the result as the average of all crimes across the entire dataset.

In [44]:
total_sum_crime= US_Crime_Rates_1960_2014_df.withColumn("AVG_Crime",
                                                    col('violent') + col('property') + col('murder') + col('forcible_rape') + 
                                                    col('robbery') + col('Aggravated_assault') + col('Burglary')+ col('Larceny_Theft')
                                                    + col('Vehicle_Theft'))

#avaerage of calculated sum
avegare_crime = total_sum_crime.agg(avg('AVG_Crime'))
print("Average of all crime:", avegare_crime.collect()[0][0])
total_sum_crime.select('Year','AVG_Crime').show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `` cannot be resolved. Did you mean one of the following? [`Year`, `Total`, `Murder`, `Robbery`, `Violent`].;
'Aggregate ['], [', avg(AVG_Crime#1823) AS avg(AVG_Crime)#1851]
+- Project [Year#943, Population#944, Total#945, Violent#946, Property#947, Murder#948, Forcible_Rape#949, Robbery#950, Aggravated_assault#951, Burglary#952, Larceny_Theft#953, Vehicle_Theft#954, ((((((((violent#946 + property#947) + murder#948) + forcible_rape#949) + robbery#950) + Aggravated_assault#951) + Burglary#952) + Larceny_Theft#953) + Vehicle_Theft#954) AS AVG_Crime#1823]
   +- Relation [Year#943,Population#944,Total#945,Violent#946,Property#947,Murder#948,Forcible_Rape#949,Robbery#950,Aggravated_assault#951,Burglary#952,Larceny_Theft#953,Vehicle_Theft#954] csv


In [ ]:
#sql

total_crime_sum = spark.sql('''
                      SELECT *,
                            (Violent + Property + Murder + Forcible_rape + Robbery +
                            Aggravated_assault + Burglary + Larceny_Theft + Vehicle_Theft) AS TotalCrimeSum
                     FROM crime_rates
                    ''')


average_crime = total_crime_sum.agg(avg('TotalCrimeSum'))
print("Average of all crime:", avegare_crime.collect()[0][0])
total_crime_sum.select('year','TotalCrimeSum').show()

Average of all crime: 21201546.145454545
+----+-------------+
|year|TotalCrimeSum|
+----+-------------+
|1960|      6768320|
|1961|      6975980|
|1962|      7504420|
|1963|      8218940|
|1964|      9129240|
|1965|      9478780|
|1966|     10446960|
|1967|     11806860|
|1968|     13440420|
|1969|     14821740|
|1970|     16196040|
|1971|     17176400|
|1972|     16497600|
|1973|     17436220|
|1974|     20506940|
|1975|     22584730|
|1976|     22699410|
|1977|     21969060|
|1978|     22417910|
|1979|     24499060|
+----+-------------+
only showing top 20 rows



### Window Functions

Question: Calculate the cumulative sum of "Property" values over the years using a window function in the US_Crime_Rates_1960_2014_df DataFrame.

In [ ]:
#pyspark
from pyspark.sql.window import Window

window_spec = Window.partitionBy("Year").rowsBetween(Window.unboundedPreceding, Window.currentRow)

US_Crime_Rates_1960_2014_df_window = US_Crime_Rates_1960_2014_df.withColumn(
    "CumulativePropertySum", sum(col("Property")).over(window_spec)
)

US_Crime_Rates_1960_2014_df_window.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|CumulativePropertySum|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|              3095700|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|              3198600|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|              3450700|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 10

In [ ]:
#sql

window_query = spark.sql('''
                    select *,
                    sum(property) OVER (order by Year ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS CumulativePropertySum
                    from crime_rates
                        ''')

window_query.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|CumulativePropertySum|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+---------------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|              3095700|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|              6294300|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|              9745000|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 10

23/09/04 09:39:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/04 09:39:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/04 09:39:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/04 09:39:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/04 09:39:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### Pivot
Question: You are working with a DataFrame named US_Crime_Rates_1960_2014_df that contains crime data for different crime types over the years. 

In [ ]:
crimes=["Violent","Property","Murder","Forcible_Rape","Robbery","Aggravated_assault","Burglary","Larceny_Theft","Vehicle_Theft"]


In [ ]:
selected_df = US_Crime_Rates_1960_2014_df.select(crimes)
selected_df.show()


+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
| 364220| 4200400|  9360|        21420| 130390|            203050| 1213200|      2514400|       472800|
| 387390| 4352000|  9960|        23410| 138690|            215330| 1282500|      2572600|       496900|
| 430180| 4793300| 11040|        25820| 157990|            23533

In [45]:

pivoted_df = selected_df.groupBy("Year").pivot(crimes).sum()
pivoted_df.show()

NameError: name 'selected_df' is not defined